In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import cv2


model_path = "lsb_mobilenetv2_model (2).h5"
model = tf.keras.models.load_model(model_path)


def predict_lsb(img_path, img_size=(224, 224)):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)

    # Normalize (same as during training)
    img_array = img_array / 224.0

    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prob = model.predict(img_array)[0][0]

    pred = 1 if prob > 0.5 else 0

    label = "LSB Embedded" if pred == 1 else "Clean Image"
    print(f"Prediction: {label}")
    print(f"Confidence: {prob:.4f}")

    return label, prob


test_image = "lsb_224.jpg"
predict_lsb(test_image)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction: Clean Image
Confidence: 0.0012


('Clean Image', np.float32(0.0011584515))

In [1]:
import cv2
import numpy as np

def embed_lsb_half_payload(input_image_path, output_image_path):
    # Load image
    img = cv2.imread(input_image_path)
    if img is None:
        raise ValueError("Image not found or invalid format.")

    # Resize to match model size (224x224)
    img = cv2.resize(img, (224, 224))

    h, w, c = img.shape
    total_bits = h * w * c
    payload_bits = int(total_bits * 0.5)  # 50% payload

    # Flatten
    flat_img = img.flatten()

    # Random bits
    data = np.random.randint(0, 2, payload_bits, dtype=np.uint8)

    # Apply LSB
    flat_img[:payload_bits] = (flat_img[:payload_bits] & 0xFE) | data

    # Restore shape
    stego_img = flat_img.reshape(img.shape)

    # Save
    cv2.imwrite(output_image_path, stego_img)
    print(f"Saved stego image: {output_image_path}")


input_path = "clean image.jpg"          # your original clean image
output_path = "lsb_224.jpg"       # the output LSB image

embed_lsb_half_payload(input_path, output_path)



Saved stego image: lsb_224.jpg
